In [10]:
import pandas as pd
import glob
import os
import string


def read_excel(file_path: str) -> pd.DataFrame:
    """
    Reads an Excel file with the specified format.
    
    Args:
        file_path (str): Path to the Excel file.
    
    Returns:
        pd.DataFrame: The DataFrame loaded from the Excel file.
    """
    return pd.read_excel(file_path, sheet_name='Respuestas de formulario 1')


def process_data(df: pd.DataFrame) -> (pd.DataFrame, str):
    """
    Processes the DataFrame by converting "Marca temporal" to a date format and cleaning email addresses.
    
    Args:
        df (pd.DataFrame): The DataFrame to process.
    
    Returns:
        pd.DataFrame: The processed DataFrame.
        str: The formatted target date.
    """
    # Convert "Marca temporal" to a date in `YYYY-MM-DD` format
    df['Marca temporal'] = pd.to_datetime(df['Marca temporal']).dt.date
    df['Marca temporal'] = df['Marca temporal'].apply(lambda x: x.strftime('%Y-%m-%d'))
    target_date = df['Marca temporal'].iloc[0]
    df['Dirección de correo electrónico'] = df['Dirección de correo electrónico'].apply(clean_email)
    return df, target_date


def match_and_update(asistencias: pd.DataFrame, df: pd.DataFrame, target_date: str) -> pd.DataFrame:
    """
    Matches emails and updates the `asistencias` DataFrame.
    
    Args:
        asistencias (pd.DataFrame): The main attendance DataFrame.
        df (pd.DataFrame): The processed DataFrame.
        target_date (str): The target date extracted from the processed DataFrame.
    
    Returns:
        pd.DataFrame: The updated `asistencias` DataFrame.
    """
    if target_date in asistencias.columns:
        # Match emails
        email_matches = asistencias.iloc[:, 3].apply(
            lambda email: 1 if email in df['Dirección de correo electrónico'].values else 0
        )
        
        # Assign results to the column in `asistencias`
        asistencias[target_date] = email_matches
        print(f"En la lista hay {len(df['Dirección de correo electrónico'].unique())} alumnos")
        print(f"Se generaron {email_matches.sum()} match entre alumnos")
    else:
        print(f"Column with date {target_date} not found in asistencias.")
    
    return asistencias


def clean_email(email: str) -> str:
    """
    Cleans an email address by removing non-printable characters and converting to lowercase.
    
    Args:
        email (str): The email to clean.
    
    Returns:
        str: The cleaned email address.
    """
    return email.strip().strip(string.whitespace).lower()

def convert_datetime_columns(df):
    # Iterate through column names and check for the format `YYYY-MM-DD 00:00:00`
    new_column_names = {}
    for col in df.columns:
        try:
            # Try converting the column name to a datetime object
            date = pd.to_datetime(col, format='%Y-%m-%d %H:%M:%S', errors='coerce')
            if pd.notna(date):  # If conversion is successful
                # Convert to the new format `YYYY-MM-DD`
                new_column_names[col] = date.strftime('%Y-%m-%d')
        except Exception:
            pass  # Skip columns that cannot be converted

    # Rename columns in the DataFrame
    df.rename(columns=new_column_names, inplace=True)

def run_pipeline(folder_path: str, asistencias: pd.DataFrame):
    """
    Main function to process all attendance Excel files and update the `asistencias` DataFrame.
    
    Args:
        folder_path (str): Path to the folder containing attendance Excel files.
        asistencias (pd.DataFrame): The main attendance DataFrame to update.
    
    Returns:
        pd.DataFrame: The updated `asistencias` DataFrame.
    """

    convert_datetime_columns(asistencias)
    asistencias.iloc[:, 3] = asistencias.iloc[:, 3].apply(clean_email)
    # Find all Excel files matching the pattern
    excel_files = glob.glob(os.path.join(folder_path, "Encuesta clase y asistencia *.xlsx"))
    
    for file_path in excel_files:
        print(f"Processing file: {file_path}")
        
        # Step 1: Read the Excel file with assistance data
        df = read_excel(file_path)
        
        # Step 2: Process assistance data
        df, target_date = process_data(df)
        
        # Step 3: Match and update with `asistencias', which is the main DataFrame with all attendance
        asistencias = match_and_update(asistencias, df, target_date)
    
    return asistencias


# Example usage
if __name__ == "__main__":
    # read `asistencias', which is the main DataFrame with all attendance
    folder_path =  '/Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/'
    asistencias = pd.read_excel(folder_path + 'mitic-data-science-team-1-septiembre-2024.xlsx',
                            sheet_name='Asistencia',header=2)
    asistencias = run_pipeline(folder_path, asistencias)
    # Optionally save the updated `asistencias`
    asistencias.to_excel('/Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/updated_asistencias.xlsx', index=False)


Processing file: /Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/Encuesta clase y asistencia 05-11-2024.xlsx
En la lista hay 39 alumnos
Se generaron 30 match entre alumnos
Processing file: /Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/Encuesta clase y asistencia 19-11-2024.xlsx
En la lista hay 35 alumnos
Se generaron 29 match entre alumnos
Processing file: /Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/Encuesta clase y asistencia 31-10-2024.xlsx
En la lista hay 31 alumnos
Se generaron 26 match entre alumnos
Processing file: /Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/data/Encuesta clase y asistencia 24-10-2024.xlsx
En la lista hay 39 alumnos
Se generaron 33 match entre alumnos
Processing file: /Users/iairlinker/Documents/repos/mitic-data-science-team-1-septiembre-2024/automatizaciones/da